In [66]:
def siguiente_alpha(letra):
    return chr(ord(letra) + 1)

In [67]:
Ruedas = {
    "base": "ABCDEFGHIJKLMNOPQRSTUVWXYZ",
    "I"   : "EKMFLGDQVZNTOWYHXUSPAIBRCJ",
    "II"  : "AJDKSIRUXBLHWTMCQGZNPYFVOE",
    "III" : "BDFHJLCPRTXVZNYEIWGAKMUSQO",
    "IV"  : "ESOVPZJAYQUIRHXLNFTGKDCMWB",
    "V"   : "VZBRGITYUPSDNHLXAWMJQOFECK",
    "VI"  : "JPGVOUMFYQBENHZRDKASXLICTW",
    "VII" : "NZJHGRCXMYSWBOUFAIVLPEKQDT",
    "VIII": "FKQHTLXOCBJSPDZRAMEWNIUYGV"
}

Ruedas_griegas = {
    "base" : "ABCDEFGHIJKLMNOPQRSTUVWXYZ",
    "beta" : "LEYJVCNIXWPBQMDRTAKZGFUHOS",
    "gamma": "FSOKANUERHMBTIYCWLQPZXVGJD"
}

Reflectores = {
    "base": "ABCDEFGHIJKLMNOPQRSTUVWXYZ",
    "A"   : "EJMZALYXVBWFCRQUONTSPIKHGD",
    "B"   : "YRUHQSLDPXNGOKMIEBFZCWVJAT",
    "C"   : "FVPJIAOYEDRZXWGCTKUQSBNMHL"
}

In [68]:
class Rotor:
    def __init__(self, rueda: str, ringstellung: str):      
        self.wiring = rueda
        indice_ring = Ruedas["base"].find(ringstellung)

        # Aplicamos el ringstellung, moviendo cada letra ciertas posiciones con respecto a la base
        wiring_as_str = self.wiring
        
        # Los str son inmutables, así que lo cambiamos temporalmente por una lista
        self.wiring = list(self.wiring)
        
        for i in range(len(self.wiring)):
            indice = Ruedas["base"].find(self.wiring[i]) + indice_ring
            self.wiring[i] = Ruedas["base"][indice % 26]
        

        # Desplazamos la cadena indice_ring lugares a la derecha
        self.wiring = self.wiring[len(self.wiring) - indice_ring : ] + self.wiring[ : len(self.wiring) - indice_ring ]
    
        # Restauramos la cadena
        cadena = ""
        for c in self.wiring:
            cadena = cadena + c
        
        self.wiring = cadena   


    def siguiente(self, indice):
        letra_a_buscar = self.wiring[self.indice]
        indice_siguiente = Ruedas["base"].find(letra_a_buscar)
        return indice_siguiente

    def revertir(self, indice):
        letra_en_base = Ruedas["base"][indice]
        indice_de_salida = self.wiring.find(letra_en_base)    
        return indice_de_salida

In [69]:
class Reflector:
    def __init__(self, reflector: str):
        self.wiring = reflector
    
    def reflejar(self, indice):
        """
        `Indice` representa la letra de base[indice].
        """
        letra_a_buscar = self.wiring[self.indice]
        indice_siguiente = Reflectores["base"].find(letra_a_buscar)
        return indice_siguiente

In [70]:
class Plugboard:
    def __init__(self, parametros):
        """
        El parámetro `mapping` es una lista de duplas, de la forma
        ```
        [(letra1, letra2), (letra3, letra4)]
        ```
        
        De forma que se construirá un mapa tal que letra1 -> letra2, etc...
        """
        
        self.base = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        self.mapping = {}

        # Nos guardamos primero un mapa de la forma A -> A, B -> B, ...
        for letra in self.base:
            self.mapping[letra] = letra

        # Y ahora lo sustituimos por lo que nos han mandado
        for dupla in parametros:
            self.mapping[dupla[0]] = dupla[1]
            self.mapping[dupla[1]] = dupla[0]
    
    def siguiente(self, letra: str):
        return self.mapping[letra]

In [85]:
class Enigma:
    def __init__(self, ruedas: list[str], ringstellungs: str, grundstellungs: str, reflector: str, steckern: list[(str, str)]):
        if len(ruedas) != len(ringstellungs):
            print("Me has dado un número de ruedas distinto de la configuración del ringstellung")
        if len(ruedas) != len(grundstellungs):
            print("Me has dado un número de ruedas distinto de la configuración del grundstellung")
        
        self.rotores = []
        for rotor, ring in zip(ruedas, ringstellungs):
            self.rotores.append(Rotor(rotor, ring))
        
        self.indices = []
        for letra in grundstellungs:
            self.indices.append(Ruedas["base"].find(letra))

        self.plugboard = Plugboard(steckern)
        self.reflector = Reflector(reflector)
    
    def cifrar_caracter(self, c: str):
        # Letra -> plugbooard -> Rotores -> Reflector -> Rotores -> plugboard -> salida
        letra = self.plugboard.siguiente(c)
        


        return letra

    def cifrar(self, mensaje: str):
        # FIXME cuidado con la configuración inicial
        
        salida = ""

        for letra in mensaje:
            if letra == " ":
                salida = salida + " "
            else:
                salida = salida + self.cifrar_caracter(letra)
        
        return salida



        


In [90]:
def es_conmutativo(mensaje1, mensaje2):
    return mensaje1 == mensaje2

In [92]:
rotores = [Ruedas["V"], Ruedas["VI"], Ruedas["VIII"], Ruedas_griegas["beta"]]
ringstellungs = "EPEL"
grundstellungs = "NAEM"
clavijas = [("A", "E"), ("B", "F"), ("C", "M"), ("D", "Q"), ("H", "U"), ("J", "N"), ("L", "X"), ("P", "R"), ("S", "Z"), ("V", "W")]

enigma = Enigma(rotores, ringstellungs, grundstellungs, Reflectores["C"], clavijas)

mensaje = "DUHF TETO LANO TCTO UARB BFPM HPHG CZXT DYGA HGUF XGEW KBLK GJWL QXXT GPJJ AVTO CKZF SLPP QIHZ FXOE BWII EKFZ LCLO AQJU LJOY HSSM BBGW HZAN VOII PYRB RTDJ QDJJ OQKC XWDN BBTY VXLY TAPG VEAT XSON PNYN QFUD BBHH VWEP YEYD OHNL XKZD NWRH DUWU JUMW WVII WZXI VIUQ DRHY MNCY EFUA PNHO TKHK GDNP SAKN UAGH JZSM JBMH VTRE QEDG XHLZ WIFU SKDQ VELN MIMI THBH DBWV HDFY HJOQ IHOR TDJD BWXE MEAY XGYQ XOHF DMYU XXNO JAZR SGHP LWML RECW WUTL RTTV LBHY OORG LGOW UXNX HMHY FAAC QEKT HSJW DUHF TETO"


es_conmutativo(mensaje, enigma.cifrar(enigma.cifrar(mensaje)))

True